## General Import

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from math import sqrt

import gc
import time
import seaborn as sns; sns.set()

import geojson
import geopandas as gpd
from fiona.crs import from_epsg
import os, json
from shapely.geometry import shape, Point, Polygon, MultiPoint
from geopandas.tools import sjoin
import matplotlib.cm as cm
import matplotlib.pyplot as plt # plotting
import seaborn as sns; sns.set()
from IPython.display import Image

import pickle
import folium


from branca.colormap import  linear
import json
import branca.colormap as cm

from tqdm.notebook import tqdm

import pickle

## Upload Results

In [ ]:
# seasonal model (baseline)
path = '../input/seasonal-persistence-model/save_predictions_results.pkl'
with open(path,'rb') as f:
        base_rmse_list, base_mae_list = pickle.load(f)
        
# Direct simple LSTM model
path = '../input/simple-lstm-tensorflow/save_predictions_results.pkl'
with open(path,'rb') as f:
        Dlstm_rmse_list, Dlstm_mae_list = pickle.load(f)
        
# Autoregressive simple LSTM model
path = '../input/simple-ar-lstm-tensorflow/save_predictions_results.pkl'
with open(path,'rb') as f:
        Alstm_rmse_list, Alstm_mae_list = pickle.load(f)

# Direct LSTM encoder decoder model
path = '../input/lstm-encoder-decoder-tensorflow/save_predictions_results.pkl'
with open(path,'rb') as f:
        DED_rmse_list, DED_mae_list = pickle.load(f)
        
# Autoregressive LSTM encoder decoder model
path = '../input/ar-lstm-encoder-decoder-tensorflow/save_predictions_results.pkl'
with open(path,'rb') as f:
        AED_rmse_list, AED_mae_list = pickle.load(f)

### compute total average metric

### Normalised RMSE metric

In [ ]:
base_rmse = np.mean(base_rmse_list)
std_base_rmse = np.std(base_rmse_list)
print(' Baseline RMSE '+str(base_rmse)+' +-'+str(std_base_rmse))
print('')

Dlstm_rmse = np.mean(Dlstm_rmse_list)
std_Dlstm_rmse = np.std(Dlstm_rmse_list)
print(' Dir LSTM model RMSE '+str(Dlstm_rmse)+' +-'+str(std_Dlstm_rmse))
print('')

Alstm_rmse = np.mean(Alstm_rmse_list)
std_Alstm_rmse = np.std(Alstm_rmse_list)
print(' AutoRegr LSTM model RMSE '+str(Alstm_rmse)+' +-'+str(std_Alstm_rmse))
print('')

DED_rmse = np.mean(DED_rmse_list)
std_DED_rmse = np.std(DED_rmse_list)
print(' Dir LSTM Enc-Dec RMSE '+str(DED_rmse)+' +-'+str(std_DED_rmse))
print('')

AED_rmse = np.mean(AED_rmse_list)
std_AED_rmse = np.std(AED_rmse_list)
print(' AutoRegr LSTM Enc-Dec RMSE '+str(AED_rmse)+' +-'+str(std_AED_rmse))
print('')

### MAE metric

In [ ]:
base_mae = np.mean(base_mae_list)
std_base_mae = np.std(base_mae_list)
print(' Baseline MAE '+str(base_mae)+' +-'+str(std_base_mae))
print('')

Dlstm_mae = np.mean(Dlstm_mae_list)
std_Dlstm_mae = np.std(Dlstm_mae_list)
print(' Dir LSTM model MAE '+str(Dlstm_mae)+' +-'+str(std_Dlstm_mae))
print('')

Alstm_mae = np.mean(Alstm_mae_list)
std_Alstm_mae = np.std(Alstm_mae_list)
print(' AutoRegr LSTM model MAE '+str(Alstm_mae)+' +-'+str(std_Alstm_mae))
print('')

DED_mae = np.mean(DED_mae_list)
std_DED_mae = np.std(DED_mae_list)
print(' Dir LSTM Enc-Dec MAE '+str(DED_mae)+' +-'+str(std_DED_mae))
print('')

AED_mae = np.mean(AED_mae_list)
std_AED_mae = np.std(AED_mae_list)
print(' AutoRegr LSTM Enc-Dec MAE '+str(AED_mae)+' +-'+str(std_AED_mae))
print('')

## compute average metric for each forecast horizon t+H

### RMSE metric

In [ ]:
base_rmse = np.mean(base_rmse_list, axis=0).mean(axis=1)
std_base_rmse = np.std(base_rmse_list, axis=0).mean(axis=1)

Dlstm_rmse = np.mean(Dlstm_rmse_list, axis=0).mean(axis=1)
std_Dlstm_rmse = np.std(Dlstm_rmse_list, axis=0).mean(axis=1)

Alstm_rmse = np.mean(Alstm_rmse_list, axis=0).mean(axis=1)
std_Alstm_rmse = np.std(Alstm_rmse_list, axis=0).mean(axis=1)

DED_rmse = np.mean(DED_rmse_list, axis=0).mean(axis=1)
std_DED_rmse = np.std(DED_rmse_list, axis=0).mean(axis=1)

AED_rmse = np.mean(AED_rmse_list, axis=0).mean(axis=1)
std_AED_rmse = np.std(AED_rmse_list, axis=0).mean(axis=1)

In [ ]:
ax = ['t+1', 't+2', 't+3', 't+4', 't+5', 't+6', 't+7', 't+8','t+9','t+10','t+11','t+12']


mean_rmse = pd.DataFrame(list(zip(Dlstm_rmse/base_rmse, Alstm_rmse/base_rmse, DED_rmse/base_rmse, AED_rmse/base_rmse)), 
               columns = ['D_lstm', 'A_lstm', 'D_lstm_ED', 'A_lstm_ED'], index=ax)


fig, ax = plt.subplots(figsize=(20, 10))
mean_rmse.plot.bar( ax=ax, capsize=3, rot=45, grid=True, color=['blue', 'purple', 'orange','red']) #yerr=std_rmse,
fig.suptitle('Normalized Multi-horizon RMSE', fontsize=20)
ax.legend(loc='center right')
plt.show()



In [ ]:
mean_rmse_scatter = mean_rmse.reset_index()

positions = (0,1, 2, 3,4,5,6,7,8,9,10,11)
labels = ('t+1', 't+2', 't+3', 't+4', 't+5', 't+6', 't+7', 't+8','t+9','t+10','t+11','t+12')


fig, ax = plt.subplots(figsize=(20, 10))

plt.plot( mean_rmse_scatter['D_lstm'],'o', label = 'D_lstm', c ='orange', markersize=10)
plt.plot( mean_rmse_scatter['D_lstm'], '--', c ='orange', linewidth=2.5)

plt.plot( mean_rmse_scatter['A_lstm'],'o', label = 'A_lstm', c ='green', markersize=10)
plt.plot( mean_rmse_scatter['A_lstm'], '--', c ='green', linewidth=2.5)

plt.plot( mean_rmse_scatter['D_lstm_ED'],'o', label = 'D_lstm_ED', c ='blue', markersize=10)
plt.plot( mean_rmse_scatter['D_lstm_ED'], '--', c ='blue', linewidth=2.5)

plt.plot( mean_rmse_scatter['A_lstm_ED'],'o', label = 'A_lstm_ED', c ='purple', markersize=10)
plt.plot( mean_rmse_scatter['A_lstm_ED'], '--', c ='purple', linewidth=2.5)

plt.xticks(positions, labels, fontsize=25, rotation=30)
plt.title('Normalized Multi-horizon RMSE', fontsize=30)
ax.legend(loc='upper left', fontsize=30)
plt.savefig("rmse.png")
plt.show()

### MAE metric

In [ ]:
base_mae = np.mean(base_mae_list, axis=0).mean(axis=1)
std_base_mae = np.std(base_mae_list, axis=0).mean(axis=1)

Dlstm_mae = np.mean(Dlstm_mae_list, axis=0).mean(axis=1)
std_Dlstm_mae = np.std(Dlstm_mae_list, axis=0).mean(axis=1)

Alstm_mae = np.mean(Alstm_mae_list, axis=0).mean(axis=1)
std_Alstm_mae = np.std(Alstm_mae_list, axis=0).mean(axis=1)

DED_mae = np.mean(DED_mae_list, axis=0).mean(axis=1)
std_DED_mae = np.std(DED_mae_list, axis=0).mean(axis=1)

AED_mae = np.mean(AED_mae_list, axis=0).mean(axis=1)
std_AED_mae = np.std(AED_mae_list, axis=0).mean(axis=1)

In [ ]:
ax = ['t+1', 't+2', 't+3', 't+4', 't+5', 't+6', 't+7', 't+8','t+9','t+10','t+11','t+12']


mean_mae = pd.DataFrame(list(zip( Dlstm_mae/base_mae, Alstm_mae/base_mae, DED_mae/base_mae, AED_mae/base_mae)), 
               columns = ['D_lstm', 'A_lstm', 'D_lstm_ED', 'A_lstm_ED'], index=ax)


# std_mae = pd.DataFrame(list(zip(std_base_mae, std_Dlstm_mae, std_Alstm_mae, std_DED_mae, std_AED_mae)),
#                    columns = ['Base', 'D_lstm', 'A_lstm','D_lstm_ED', 'A_lstm_ED'],index=ax)


fig, ax = plt.subplots(figsize=(20, 8))
mean_rmse.plot.bar( ax=ax, capsize=3, rot=45, grid=True, color=['lightgreen', 'yellow', 'orange','red']) #yerr=std_rmse,
fig.suptitle('Normalized Multi-horizon MAE', fontsize=20)
ax.legend(loc='center right')
plt.show()


In [ ]:
mean_mae_scatter = mean_mae.reset_index()

positions = (0,1, 2, 3,4,5,6,7,8,9,10,11)
labels = ('t+1', 't+2', 't+3', 't+4', 't+5', 't+6', 't+7', 't+8','t+9','t+10','t+11','t+12')


fig, ax = plt.subplots(figsize=(20, 10))

plt.plot( mean_mae_scatter['D_lstm'],'o', label = 'D_lstm', c ='orange', markersize=10)
plt.plot( mean_mae_scatter['D_lstm'], '--', c ='orange', linewidth=2.5)

plt.plot( mean_mae_scatter['A_lstm'],'o', label = 'A_lstm', c ='green', markersize=10)
plt.plot( mean_mae_scatter['A_lstm'], '--', c ='green', linewidth=2.5)

plt.plot( mean_mae_scatter['D_lstm_ED'],'o', label = 'D_lstm_ED', c ='blue', markersize=10)
plt.plot( mean_mae_scatter['D_lstm_ED'], '--', c ='blue', linewidth=2.5)

plt.plot( mean_mae_scatter['A_lstm_ED'],'o', label = 'A_lstm_ED', c ='purple', markersize=10)
plt.plot( mean_mae_scatter['A_lstm_ED'], '--', c ='purple', linewidth=2.5)

plt.xticks(positions, labels, fontsize=25, rotation=30)
plt.title('Normalized Multi-horizon MAE', fontsize=30)
ax.legend(loc='upper left', fontsize=30)
plt.savefig("mae.png")
plt.show()